In [1]:
!pip  install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python matplotlib

  Created wheel for wrapt: filename=wrapt-1.12.1-py3-none-any.whl size=19558 sha256=e78e071b252d33c1fa00d6a434775df85fc5070a36f9733491ab401b3e762514
  Stored in directory: c:\users\asus\appdata\local\pip\cache\wheels\5f\fd\9e\b6cf5890494cb8ef0b5eaff72e5d55a70fb56316007d6dfe73
Successfully built wrapt
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.11.2
    Uninstalling wrapt-1.11.2:
      Successfully uninstalled wrapt-1.11.2
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.42.0
    Uninstalling grpcio-1.42.0:
      Successfully uninstalled grpcio-1.42.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.0.0
    Uninstalling absl-py-1.0.0:
      Successfully uninstalled absl-py-1.0.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-est

In [1]:
import tensorflow as tf 
import numpy as np 
from matplotlib import pyplot as plt 
import cv2

In [2]:
interpreter = tf.lite.Interpreter(model_path='lite-model_movenet_singlepose_lightning_3.tflite')
interpreter.allocate_tensors()

**Draw Keypoints**

In [4]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped: 
        ky, kx, kp_conf = kp 
        if kp_conf > confidence_threshold: 
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1)

**Draw Edges**

In [5]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [6]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1,p2 = edge 
        y1, x1, c1 = shaped[p1] 
        y2, x2, c2 = shaped[p2]
        
        if(c1 > confidence_threshold) & (c2 > confidence_threshold):
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255),2)

In [8]:
cap = cv2.VideoCapture(0) #connects to webcam
while cap.isOpened(): #loop through every frame
    ret, frame = cap.read() 
    
    #Reshape 
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img,axis=0), 192, 192)
    input_image = tf.cast(img, dtype=tf.float32)
    # Setup input and output
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    # Make predictions
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    print(keypoints_with_scores)
    
    # rendering
    draw_connections(frame, keypoints_with_scores, EDGES, 0.4)
    draw_keypoints(frame, keypoints_with_scores, 0.4)
    
    cv2.imshow('MoveNet Lightning', frame) #rendering to the screen 
    
    if cv2.waitKey(10) & 0xFF==ord('q'): 
        break 
cap.release()
cv2.destroyAllWindows()

[[[[0.45201713 0.4450633  0.6956281 ]
   [0.4014374  0.48601672 0.72438717]
   [0.41575    0.3915139  0.5222204 ]
   [0.4233269  0.55782604 0.659299  ]
   [0.46218264 0.3365714  0.6190213 ]
   [0.7018315  0.74125504 0.8072972 ]
   [0.7350446  0.25505716 0.721639  ]
   [0.9656652  0.86854744 0.14678967]
   [0.9606949  0.13869959 0.06883734]
   [0.98344874 0.8925459  0.02850387]
   [0.8223457  0.3054081  0.00841758]
   [1.0183748  0.73598874 0.01696312]
   [0.31450036 0.30595273 0.04447505]
   [0.7196405  0.79097664 0.01793283]
   [0.74364066 0.30613807 0.0056251 ]
   [0.8532851  0.9863086  0.00964764]
   [0.87512743 0.1429827  0.00658858]]]]
[[[[0.4491826  0.439334   0.5927268 ]
   [0.39901394 0.48182684 0.57817376]
   [0.41416115 0.38637877 0.5068902 ]
   [0.42157847 0.5541881  0.70594734]
   [0.46195203 0.33264053 0.5398518 ]
   [0.6943378  0.7446529  0.6867467 ]
   [0.7449062  0.25216624 0.5654976 ]
   [0.9661597  0.8707661  0.1390653 ]
   [0.9598531  0.14766055 0.05131802]
   [0.886